In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Stat models
from statsmodels.tsa.vector_ar.var_model import VAR

# Functions
sys.path.append(os.path.abspath('functions'))
from preprocessing import read_csv_files, check_missing_values, forward_fill, backward_fill, linear_interpolation, calculate_volatility, adf_test, check_date_range, extract_date_range, map_date_range, create_volatility_df

## Data Preprocessing

In [2]:
# Retrieve a list of DataFrames
dataframes = read_csv_files("data")

In [3]:
# Convert each column in every DataFrame to lowercase
for key in dataframes:
    dataframes[key].columns = map(str.lower, dataframes[key].columns)

Converting the `date` column to datetime object, this process has to be handled individually since each DataFrame has a different date format. We cannot let Pandas infers the date format for each DataFrame since it can be prone to infer the wrong format. Below are the format of each DataFrame:
- Philippines: MM/DD/YYYY
- Singapore: MM/DD/YYYY
- India: YYYY-MM-DD
- United Kingdom: DD/MM/YYYY
- Mexico: YYYY-MM-DD
- Japan: YYYY-MM-DD
- Vietnam: DD/MM/YYYY
- Korea: YYYY-MM-DD
- Thailand: YYYY-MM-DD
- Brazil: YYYY-MM-DD
- Malaysia: DD/MM/YYYY
- Switzerland: YYYY-MM-DD
- China: DD/MM/YYYY
- Russia: YYYY-MM-DD
- United States: YYYY-MM-DD

In [4]:
date_format_mapping = {
  'philippines': '%m/%d/%Y',
  'singapore': '%m/%d/%Y',
  'india': '%Y-%m-%d',
  'uk': '%d/%m/%Y',
  'mexico': '%Y-%m-%d',
  'japan': '%Y-%m-%d',
  'vietnam': '%d/%m/%Y',
  'korea': '%Y-%m-%d',
  'thailand': '%Y-%m-%d',
  'brazil': '%Y-%m-%d',
  'malaysia': '%d/%m/%Y',
  'switzerland': '%Y-%m-%d',
  'china': '%d/%m/%Y',
  'russia': '%Y-%m-%d',
  'us': '%Y-%m-%d',
}

# Convert the date columns to datetime objects
for key in dataframes:
    try:
      dataframes[key]['date'] = pd.to_datetime(
        dataframes[key]['date'], 
        format=date_format_mapping[key]
      )
    except Exception as e:
      print(f"Error occurred for country: {key}")
      print(f"Error message: {str(e)}")

In [5]:
# Sort the dataframes by date in ascending order
for key in dataframes:
    dataframes[key] = dataframes[key].sort_values(by='date')

In [6]:
# Reset the index of the dataframes
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

In [7]:
# Extract only open, high, low, close columns
for key in dataframes:
    dataframes[key] = dataframes[key][['date', 'open', 'high', 'low', 'close']]

In [8]:
for df in dataframes.values():
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3055 entries, 0 to 3054
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3055 non-null   datetime64[ns]
 1   open    3055 non-null   object        
 2   high    3055 non-null   object        
 3   low     3055 non-null   object        
 4   close   3055 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 119.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3181 entries, 0 to 3180
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3181 non-null   datetime64[ns]
 1   open    3181 non-null   object        
 2   high    3181 non-null   object        
 3   low     3181 non-null   object        
 4   close   3181 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 124.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Ra

Since the columns open, high, low, close have different dtypes (`float64` and `object`) for different DataFrame, they should be converted to `float64`.

In [9]:
# Convert open, high, low, close columns to float
for key in dataframes:
  for col in ['open', 'high', 'low', 'close']:
    if dataframes[key][col].dtype == 'object':
      dataframes[key][col] = dataframes[key][col].str.replace(',', '').astype(float)


In [10]:
# Check whether the columns have been converted to float
for df in dataframes.values():
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3055 entries, 0 to 3054
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3055 non-null   datetime64[ns]
 1   open    3055 non-null   float64       
 2   high    3055 non-null   float64       
 3   low     3055 non-null   float64       
 4   close   3055 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 119.5 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3181 entries, 0 to 3180
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3181 non-null   datetime64[ns]
 1   open    3181 non-null   float64       
 2   high    3181 non-null   float64       
 3   low     3181 non-null   float64       
 4   close   3181 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 124.4 KB
None
<class 'pandas.core.frame.DataFrame'>
Ra

##### Data imputation

This section aims to handle the missing values present in each DataFrame. Since these stock datasets will eventually be fed into a Vector Auto-regressive (VAR) model, it is quite important to choose a data imputation method that will preserve the temporal and cross-sectional relationships among countries.

Data imputation will be performed in two separate sections since one of our following step will create more missing values. A suitable method is chosen for each step, details are below:

1. Using linear interpolation to fill in the NA values originally presented in each dataset.
2. Dividing the imputed datasets from step 1 into 5 windows as defined in the Window Extraction step, resulting in 5 dictionaries containing DataFrames of countries with available data in each period.
3. The goal of this step is to normalize the date range in each time window since every country has their own public holidays, making the number of available daily trading data different for each country. Performing this step will ensure that holidays data will be included and the number of daily data available for each country within a time window is the same. For each DataFrame in a time window, map the available data into a new empty DataFrame with a date column containing every business days (excluding public holidays) of the associated period. Then, missing data from the mapped DataFrame will be imputed using forward filling method.
4. Calculate daily volatility for each DataFrame in every time window.
5. Create a single DataFrame for each time window with each column representing a country's volatility calculation.


In [11]:
check_missing_values(dataframes)

Missing values for philippines:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for singapore:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for india:
date      0
open     39
high     39
low      39
close    39
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date      0
open     49
high     49
low      49
close    49
dtype: int64


Missing values for japan:
date      0
open     93
high     93
low      93
close    93
dtype: int64


Missing values for vietnam:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date      0
open     66
high     66
low      66
close    66
dtype: int64


Missing values for thailand:
date      0
open     73
high     73
low      73
close    73
dtype: int64


Missing values for brazil:
date       0
open     462
high     462
low      462
close    462
dtype:

In [12]:
# Impute missing values in the dataframes using linear interpolation method
dataframes = linear_interpolation(dataframes)

In [13]:
# Check whether the missing values have been imputed
check_missing_values(dataframes)

Missing values for philippines:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for singapore:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for india:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for vietnam:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for brazil:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for malaysi

##### Window extraction

In this section, I will perform window extraction for the following time period:
- **Window 1:** 02.01.2002 to 17.09.2007
- **Window 2:** 18.09.2007 to 27.10.2011 (India's dataset starts from 18.09.2007)
- **Window 3:** 28.10.2011 to 31.12.2018 (Philippines's dataset starts from 28.10.2011)
- **Window 4:** 02.01.2019 to 31.12.2022 
- **Window 5:** 02.01.2023 to 30.04.2024 

##### 02.01.2002 - 17.09.2007: Recovery period from Dot-com Bubble

In [14]:
# Check whether each DataFrame contains data from 02.01.2002 to 17.09.2007
window_1_start_date = '2002-01-02'
window_1_end_date = '2007-09-17'
check_date_range(dataframes, window_1_start_date, window_1_end_date)

{'philippines': False,
 'singapore': False,
 'india': False,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': False,
 'korea': True,
 'thailand': True,
 'brazil': False,
 'malaysia': False,
 'switzerland': True,
 'china': True,
 'russia': False,
 'us': True}

The following countries do not have data available for this time period:
1. Philippine
2. Singapore
3. India
4. Vietnam
5. Brazil
6. Malaysia
7. Russia

These 7 countries will be be considered in VAR experiment for this time period, with the addition of China since it only has data from 2005.

In [15]:
dfs_window_1 = extract_date_range(dataframes, window_1_start_date, window_1_end_date)

# Check whether each DataFrame contains data from 02.01.2002 to 17.09.2007
dfs_window_1

{'philippines': Empty DataFrame
 Columns: [date, open, high, low, close]
 Index: [],
 'singapore': Empty DataFrame
 Columns: [date, open, high, low, close]
 Index: [],
 'india': Empty DataFrame
 Columns: [date, open, high, low, close]
 Index: [],
 'uk':            date    open    high     low   close
 0    2002-01-02  5217.4  5261.9  5195.2  5218.3
 1    2002-01-03  5218.3  5328.4  5218.3  5318.8
 2    2002-01-04  5318.8  5362.3  5313.9  5323.8
 3    2002-01-07  5323.8  5354.2  5277.7  5293.6
 4    2002-01-08  5293.6  5307.3  5238.7  5250.4
 ...         ...     ...     ...     ...     ...
 1438 2007-09-11  6134.1  6280.7  6134.1  6280.7
 1439 2007-09-12  6280.7  6317.1  6232.0  6306.2
 1440 2007-09-13  6306.2  6374.5  6280.2  6363.9
 1441 2007-09-14  6363.9  6363.9  6209.1  6289.3
 1442 2007-09-17  6289.3  6289.3  6168.0  6182.8
 
 [1443 rows x 5 columns],
 'mexico':            date          open          high           low         close
 0    2002-01-02   6386.180176   6415.850098   6

In [16]:
# Remove countries that do not have data for the specified date range (02.01.2002 to 17.09.2007) from the dictionary
for key in list(dfs_window_1.keys()):
  if dfs_window_1[key].empty:
    del dfs_window_1[key]

# Remove the 'china' DataFrame from the dictionary
del dfs_window_1['china']

In [17]:
len(dfs_window_1)

7

In [18]:
# Map dfs_window_1 to a new date range
dfs_window_1 = map_date_range(dfs_window_1, window_1_start_date, window_1_end_date)

In [19]:
# Check for missing values in the dfs_window_1
check_missing_values(dfs_window_1)

Missing values for uk:
date      0
open     46
high     46
low      46
close    46
dtype: int64


Missing values for mexico:
date     0
open     5
high     5
low      5
close    5
dtype: int64


Missing values for japan:
date      0
open     13
high     13
low      13
close    13
dtype: int64


Missing values for korea:
date      0
open     21
high     21
low      21
close    21
dtype: int64


Missing values for thailand:
date      0
open     27
high     27
low      27
close    27
dtype: int64


Missing values for switzerland:
date      0
open     10
high     10
low      10
close    10
dtype: int64


Missing values for us:
date      0
open     52
high     52
low      52
close    52
dtype: int64




In [20]:
# Impute missing values in dfs_window_1 using forward fill method
dfs_window_1 = forward_fill(dfs_window_1)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [21]:
# Check for missing values in the dfs_window_1
check_missing_values(dfs_window_1)

Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     2
high     2
low      2
close    2
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for switzerland:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for us:
date     0
open     0
high     0
low      0
close    0
dtype: int64




Japan still has 2 missing values. These missing values are probably the first two days of the period. Let's use backward fill to handle these two missing values.

In [22]:
dfs_window_1 = backward_fill(dfs_window_1)

/Users/annguyen/Documents/Master Degree/Dissertation/Submissions/Experiments/functions/preprocessing.py:54: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)


In [23]:
# Check for missing values in the dfs_window_1 again to make sure the missing values have been imputed
check_missing_values(dfs_window_1)

Missing values for uk:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for mexico:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for japan:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for korea:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for thailand:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for switzerland:
date     0
open     0
high     0
low      0
close    0
dtype: int64


Missing values for us:
date     0
open     0
high     0
low      0
close    0
dtype: int64




In [24]:
# Calculate the volatility for each country
dfs_window_1 = calculate_volatility(dfs_window_1)

In [25]:
# Create volatility DataFrame for window 1
df_window_1 = create_volatility_df(dfs_window_1)

In [26]:
# Print out the first 5 rows of the volatility DataFrame
df_window_1.head()

,date,uk,mexico,japan,korea,thailand,switzerland,us
0,2002-01-02,17.955641,5.943942,19.791884,51.994083,12.228153,20.881106,21.664973
1,2002-01-03,18.456913,21.957613,19.791884,25.862243,17.854374,12.890345,8.171472
2,2002-01-04,12.755441,10.922138,19.791884,21.446678,16.412695,11.019224,13.936110
3,2002-01-07,19.064627,14.299720,16.900705,41.700360,18.717009,17.311078,14.120694
4,2002-01-08,15.443547,10.420974,17.203246,22.994618,12.864248,12.207484,11.487229


Performing ADF Stationarity Test

In [27]:
for country in df_window_1.drop(columns=['date']).columns:
    adf_test(df_window_1, country)

Results of Augmented Dickey-Fuller Test for uk:
ADF Statistic: -3.312306568085512
p-value: 0.014337253065932897
Critical Values:
	1%: -3.434803261722705
	5%: -2.863506823125904
	10%: -2.567817117325164
Reject the null hypothesis. The time series uk is stationary.


Results of Augmented Dickey-Fuller Test for mexico:
ADF Statistic: -4.9554957820537355
p-value: 2.7162992260244777e-05
Critical Values:
	1%: -3.4347941822385923
	5%: -2.863502816068378
	10%: -2.5678149835134185
Reject the null hypothesis. The time series mexico is stationary.


Results of Augmented Dickey-Fuller Test for japan:
ADF Statistic: -2.976810034469136
p-value: 0.03711753950882737
Critical Values:
	1%: -3.4348184769200056
	5%: -2.86351353803652
	10%: -2.567820693109295
Reject the null hypothesis. The time series japan is stationary.


Results of Augmented Dickey-Fuller Test for korea:
ADF Statistic: -3.968076683867417
p-value: 0.001587211913799357
Critical Values:
	1%: -3.434806296467238
	5%: -2.8635081624499494
	10

##### 18.09.2007 - 27.10.2011: Global Financial Crisis

In [28]:
# Check whether each DataFrame contains data from 18.09.2007 to 27.10.2011
window_2_start_date = '2007-09-18'
window_2_end_date = '2011-10-27'
check_date_range(dataframes, window_2_start_date, window_2_end_date)

{'philippines': False,
 'singapore': True,
 'india': True,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': True,
 'korea': True,
 'thailand': True,
 'brazil': True,
 'malaysia': True,
 'switzerland': True,
 'china': True,
 'russia': False,
 'us': True}

Most countries have data from 18.09.2007 to 27.10.2011, with the exception of Philippines and Russia. In addition, there are Singapore, which only has data from 07.03.2011. These 3 countries will be remove from this time window.

In [29]:
dfs_window_2 = extract_date_range(dataframes, window_2_start_date, window_2_end_date)

# Check whether each DataFrame contains data from 18.09.2007 to 27.10.2011
dfs_window_2

{'philippines': Empty DataFrame
 Columns: [date, open, high, low, close]
 Index: [],
 'singapore':           date     open     high      low    close
 0   2011-03-07  3054.43  3074.93  3047.59  3066.52
 1   2011-03-08  3063.58  3105.90  3062.52  3103.84
 2   2011-03-09  3110.54  3113.02  3087.84  3092.90
 3   2011-03-10  3093.79  3094.38  3070.55  3075.44
 4   2011-03-11  3053.07  3058.27  3024.17  3043.49
 ..         ...      ...      ...      ...      ...
 158 2011-10-20  2719.26  2720.73  2685.34  2694.01
 159 2011-10-21  2702.51  2719.12  2698.61  2712.41
 160 2011-10-24  2745.69  2776.94  2745.50  2760.95
 161 2011-10-25  2765.18  2780.24  2752.56  2769.94
 162 2011-10-27  2776.76  2862.01  2764.82  2847.57
 
 [163 rows x 5 columns],
 'india':            date         open         high          low        close
 0    2007-09-18  6136.330078  6199.839844  6117.209961  6192.879883
 1    2007-09-19  6274.509766  6403.439941  6274.509766  6399.479980
 2    2007-09-20  6408.549805  6463

In [30]:
# Remove countries that do not have data for the specified date range (02.01.2002 to 17.09.2007) from the dictionary
for key in list(dfs_window_2.keys()):
  if dfs_window_2[key].empty:
    del dfs_window_2[key]

# Remove the 'singapore' DataFrame from the dictionary
del dfs_window_2['singapore']

In [31]:
len(dfs_window_2)

12

##### 28.10.2011 - 31.12.2018: Recovery from Global Financial Crisis

In [32]:
# Check whether each DataFrame contains data from 28.10.2011 to 31.12.2018
window_3_start_date = '2011-10-28'
window_3_end_date = '2018-12-31'
check_date_range(dataframes, window_3_start_date, window_3_end_date)

{'philippines': True,
 'singapore': True,
 'india': True,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': True,
 'korea': True,
 'thailand': True,
 'brazil': True,
 'malaysia': True,
 'switzerland': True,
 'china': True,
 'russia': True,
 'us': True}

Most DataFrame appears to have data from 28.10.2011 to 31.12.2018, except from Russia, which only has data from 2013. Russia will be removed from this time window.

In [33]:
dfs_window_3 = extract_date_range(dataframes, window_3_start_date, window_3_end_date)

# Check whether each DataFrame contains data from 28.10.2011 - 31.12.2018
dfs_window_3

{'philippines':            date     open     high      low    close
 0    2011-10-28  4335.08  4345.75  4326.25  4333.72
 1    2011-11-02  4332.93  4332.93  4254.44  4260.41
 2    2011-11-03  4264.20  4286.26  4207.44  4210.25
 3    2011-11-04  4229.72  4275.70  4229.72  4271.72
 4    2011-11-08  4277.70  4322.26  4277.70  4314.67
 ...         ...      ...      ...      ...      ...
 1738 2018-12-20  7514.07  7584.44  7498.19  7563.41
 1739 2018-12-21  7536.87  7548.13  7452.24  7479.71
 1740 2018-12-26  7436.73  7453.22  7378.88  7450.01
 1741 2018-12-27  7465.04  7514.33  7447.22  7482.66
 1742 2018-12-28  7497.16  7507.29  7466.02  7466.02
 
 [1743 rows x 5 columns],
 'singapore':            date     open     high      low    close
 163  2011-10-28  2879.73  2905.72  2865.86  2905.72
 164  2011-10-31  2903.97  2903.97  2855.77  2855.77
 165  2011-11-01  2832.37  2843.58  2785.23  2789.35
 166  2011-11-02  2760.73  2836.75  2754.69  2834.75
 167  2011-11-03  2801.57  2813.48  2781.27

In [34]:
del dfs_window_3['russia']

In [35]:
len(dfs_window_3)

14

##### 02.01.2019 - 31.12.2022: COVID-19 pandemic

In [36]:
# Check whether each DataFrame contains data from 02.01.2019 to 31.12.2022
window_4_start_date = '2019-01-02'
window_4_end_date = '2022-12-31'
check_date_range(dataframes, window_4_start_date, window_4_end_date)

{'philippines': True,
 'singapore': True,
 'india': True,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': True,
 'korea': True,
 'thailand': True,
 'brazil': True,
 'malaysia': True,
 'switzerland': True,
 'china': True,
 'russia': True,
 'us': True}

In [37]:
dfs_window_4 = extract_date_range(dataframes, window_4_start_date, window_4_end_date)

# Check whether each DataFrame contains data from 02.01.2019 to 31.12.2022
dfs_window_4

{'philippines':            date     open     high      low    close
 1743 2019-01-02  7496.57  7540.26  7465.76  7489.20
 1744 2019-01-03  7507.48  7680.60  7490.28  7680.60
 1745 2019-01-04  7657.18  7801.50  7657.18  7761.11
 1746 2019-01-07  7800.88  7900.70  7787.66  7787.66
 1747 2019-01-08  7820.26  7826.08  7702.12  7702.12
 ...         ...      ...      ...      ...      ...
 2724 2022-12-23  6567.92  6567.92  6536.67  6541.03
 2725 2022-12-26  6541.03  6541.03  6541.03  6541.03
 2726 2022-12-27  6549.09  6579.26  6519.90  6564.90
 2727 2022-12-28  6567.13  6576.71  6530.27  6566.54
 2728 2022-12-29  6553.67  6591.28  6529.69  6566.39
 
 [986 rows x 5 columns],
 'singapore':            date     open     high      low    close
 1975 2019-01-02  3072.99  3081.01  3028.84  3038.89
 1976 2019-01-03  3022.27  3041.03  3002.40  3012.88
 1977 2019-01-04  2994.02  3059.23  2993.42  3059.23
 1978 2019-01-07  3101.26  3104.32  3084.88  3102.80
 1979 2019-01-08  3108.38  3126.64  3099.33 

##### 02.01.2023 - 30.04.2024: Recovery from COVID-19 pandemic

In [38]:
# Check whether each DataFrame contains data from 02.01.2023 to 30.04.2024
window_5_start_date = '2023-01-02'
window_5_end_date = '2024-04-30'
check_date_range(dataframes, window_5_start_date, window_5_end_date)

{'philippines': True,
 'singapore': True,
 'india': True,
 'uk': True,
 'mexico': True,
 'japan': True,
 'vietnam': True,
 'korea': True,
 'thailand': True,
 'brazil': True,
 'malaysia': True,
 'switzerland': True,
 'china': True,
 'russia': True,
 'us': True}

In [39]:
dfs_window_5 = extract_date_range(dataframes, window_5_start_date, window_5_end_date)

# Check whether each DataFrame contains data from 02.01.2023 to 30.04.2024
dfs_window_5

{'philippines':            date     open     high      low    close
 2729 2023-01-03  6555.40  6624.76  6550.17  6586.01
 2730 2023-01-04  6601.50  6755.27  6601.50  6718.50
 2731 2023-01-05  6738.93  6767.05  6730.52  6761.33
 2732 2023-01-06  6736.65  6736.65  6666.68  6667.97
 2733 2023-01-09  6692.30  6790.24  6685.07  6790.24
 ...         ...      ...      ...      ...      ...
 3050 2024-04-24  6538.98  6593.18  6538.98  6572.75
 3051 2024-04-25  6578.20  6589.89  6574.88  6574.88
 3052 2024-04-26  6557.95  6628.75  6547.76  6628.75
 3053 2024-04-29  6629.57  6769.64  6629.57  6769.64
 3054 2024-04-30  6749.82  6776.54  6700.49  6700.49
 
 [326 rows x 5 columns],
 'singapore':            date     open     high      low    close
 2851 2023-01-03  3243.99  3252.12  3212.63  3245.80
 2852 2023-01-04  3251.42  3256.37  3238.70  3242.46
 2853 2023-01-05  3264.53  3297.34  3262.38  3292.66
 2854 2023-01-06  3290.32  3290.32  3271.08  3276.72
 2855 2023-01-09  3311.67  3343.47  3291.04 